In [1]:
import sys
!{sys.executable} -m pip install openpyxl

import pandas as pd
import numpy as np


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
SAMPLE_DATA_FILE = "./data/output/sample_de/sample_dane_puste_przebiegi.xlsx"
SAMPLE_EMPTY_PASSAGE_FILE = "./data/output/sample_de/sample_empty_passage_edges.xlsx"

In [3]:
data_df = pd.read_excel(SAMPLE_DATA_FILE)
data_df

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,27070/2025,61,PO2PY63,(DE) Pritzwalk - (HU) Csomor,pritzwalk,16928,de,csömör,2141,hu,2025-09-08 07:00:00,2025-09-08 06:47:13,2025-09-09 07:30:00,2025-09-09 16:25:00,137.687,1180.231,1317.918
1,27356/2025,35,PO2PY63,(HU) Csömör - (DE) CRAILSHEIM,csömör,2141,hu,crailsheim,74564,de,2025-09-10 04:00:00,2025-09-10 14:14:18,2025-09-11 14:00:00,2025-09-11 15:25:00,3.267,898.686,901.953
2,27576/2025,61,PO2PY63,(DE) CRAILSHEIM - (DE) Landsberg,crailsheim,74564,de,landsberg,06188,de,2025-09-11 08:00:00,2025-09-11 15:25:14,2025-09-12 18:00:00,2025-09-15 08:00:00,0.000,383.061,383.061
3,27165/2025,35,PO2PY67,(FR) amiens - (DE) KIEL,amiens,80013,fr,kiel,24109,de,2025-09-08 17:00:00,2025-09-08 18:30:00,2025-09-10 06:00:00,2025-09-10 08:35:00,199.784,899.613,1099.397
4,27393/2025,47,PO2PY67,(DE) Hamburg - (PL) OSIECZNA,hamburg,21129,de,osieczna,64-113,pl,2025-09-10 09:00:00,2025-09-10 13:00:00,2025-09-11 18:00:00,2025-09-11 19:28:47,101.892,592.303,694.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,27283/2025,61,WPR1249U,(PL) Łódź - (DE) BERLIN,łódź,94-406,pl,berlin,12099,de,2025-09-09 17:00:00,2025-09-10 03:06:00,2025-09-10 06:00:00,2025-09-10 12:02:35,0.000,517.388,517.388
356,27479/2025,61,WPR1249U,(DE) BERLIN - (PL) Łódź,berlin,12099,de,łódź,94-406,pl,2025-09-10 10:00:00,2025-09-10 12:45:00,2025-09-10 19:00:00,2025-09-10 22:19:00,0.000,519.184,519.184
357,27485/2025,61,WPR1249U,(PL) Łódź - (DE) BERLIN,łódź,94-406,pl,berlin,12099,de,2025-09-10 21:00:00,2025-09-10 22:23:00,2025-09-11 10:00:00,2025-09-11 15:12:00,0.000,517.388,517.388
358,27493/2025,61,WPR1249U,(DE) BERLIN - (PL) Łódź,berlin,12099,de,łódź,94-406,pl,2025-09-11 10:00:00,2025-09-11 15:28:00,2025-09-11 19:00:00,2025-09-12 00:14:00,0.000,519.184,519.184


# Macierz miast i odległości - między pustymi przejazdami

In [ ]:
data_df["pickup_planned"] = pd.to_datetime(data_df["pickup_planned"], errors="coerce")
data_df["delivery_planned"] = pd.to_datetime(data_df["delivery_planned"], errors="coerce")
data_df["delivery_actual"] = pd.to_datetime(data_df["delivery_actual"], errors="coerce")
data_df["pickup_actual"] = pd.to_datetime(data_df["pickup_actual"], errors="coerce")

data_df = data_df.sort_values(["tractor", "pickup_planned"]) #.reset_index(drop=True)

# 3️⃣ Przygotowanie listy wyników
edges = []

# 4️⃣ Iteracja po grupach ciągników
for tractor, group in data_df.groupby("tractor"):
    group = group.sort_values("pickup_planned").reset_index(drop=True)
    
    # iterujemy po parach kolejnych wierszy (1..N-1)
    for i in range(1, len(group)):
        prev = group.iloc[i - 1]
        curr = group.iloc[i]

        # Sprawdzenie, czy rozładunek poprzedniego ≠ załadunek następnego
        if prev["unload_city"] != curr["load_city"]:
            # sprawdź, czy empty_km_map > 0
            if pd.notna(curr["empty_km_map"]) and curr["empty_km_map"] > 0:
                # oblicz czas między rozładunkiem a kolejnym załadunkiem
                time_diff = None
                if pd.notna(prev["delivery_actual"]) and pd.notna(curr["pickup_actual"]):
                    time_diff = curr["pickup_actual"] - prev["delivery_actual"]
                
                # Dodaj wpis do listy
                edges.append({
                    "full_number": prev["full_number"],
                    "tractor": tractor,
                    "start_city": prev["unload_city"],
                    "unload_country": prev["unload_country"],
                    "end_city": curr["load_city"],
                    "load_country": prev["load_country"],
                    "start_date_planned": prev["delivery_planned"],
                    "start_date_actual": prev["delivery_actual"],
                    "end_date_planned": curr["pickup_planned"],
                    "end_date_actual": curr["pickup_actual"],
                    "time": time_diff,
                    "empty_km_distance": curr["empty_km_map"]
                })

# 5️⃣ Utwórz nowy DataFrame z krawędziami pustych przejazdów
edges_df = pd.DataFrame(edges)

# 6️⃣ Podgląd
print(f"\nLiczba wykrytych pustych przebiegów: {edges_df.shape[0]}\n")



Liczba wykrytych pustych przebiegów: 145



In [5]:
edges_df

,full_number,tractor,start_city,unload_country,end_city,load_country,start_date_planned,start_date_actual,end_date_planned,end_date_actual,time,empty_km_distance
0,27165/2025,PO2PY67,kiel,de,hamburg,fr,2025-09-10 06:00:00,2025-09-10 08:35:00,2025-09-10 09:00:00,2025-09-10 13:00:00,0 days 04:25:00,101.892
1,27393/2025,PO2PY67,osieczna,pl,ociąż,de,2025-09-11 18:00:00,2025-09-11 19:28:47,2025-09-12 06:00:00,2025-09-12 20:50:00,1 days 01:21:13,124.774
2,27255/2025,PO2PY71,pratteln 1,ch,buggingen,de,2025-09-09 07:00:00,2025-09-09 07:00:00,2025-09-09 11:30:00,2025-09-09 13:00:00,0 days 06:00:00,49.956
3,27182/2025,PO2PY71,la maxe,fr,bürstadt,de,2025-09-10 05:30:00,2025-09-10 11:00:00,2025-09-10 08:00:00,2025-09-10 15:00:00,0 days 04:00:00,200.687
4,27045/2025,PO2PY72,mannheim,de,bürstadt,de,2025-09-09 08:00:00,2025-09-09 09:39:00,2025-09-09 14:00:00,2025-09-09 11:51:35,0 days 02:12:35,20.895
...,...,...,...,...,...,...,...,...,...,...,...,...
140,27428/2025,WGM9044L,euskirchen,de,altfeld,de,2025-09-11 10:00:00,2025-09-10 14:07:00,2025-09-10 19:00:00,2025-09-11 07:46:00,0 days 17:39:00,236.191
141,27106/2025,WGM9816L,agnadello,it,gattatico,de,2025-09-10 10:00:00,2025-09-10 09:20:00,2025-09-10 12:00:00,2025-09-10 14:04:00,0 days 04:44:00,135.916
142,27067/2025,WGM9817L,flörsheim am main,de,altfeld,fr,2025-09-09 11:30:00,2025-09-09 10:23:00,2025-09-09 18:00:00,2025-09-09 20:53:16,0 days 10:30:16,100.411
143,27420/2025,WGM9817L,bernburg,de,sülzetal,de,2025-09-12 06:00:00,2025-09-12 07:18:00,2025-09-12 08:30:00,2025-09-12 09:56:00,0 days 02:38:00,36.813


In [6]:
edges_df["time"] = edges_df["time"].astype(str)

# 7️⃣ (opcjonalnie) zapis do pliku
edges_df.to_excel(SAMPLE_EMPTY_PASSAGE_FILE, index=False)
print(f"Zapisano wynik do: {SAMPLE_EMPTY_PASSAGE_FILE}")

Zapisano wynik do: ./data/output/sample_de/sample_empty_passage_edges.xlsx
